In [1]:
import pandas as pd
k2 = pd.read_pickle('data\\k2\\k2_data_treated.pkl')
k2_classification = k2[k2['classification'].isin(['planet', 'not planet'])]
x_k2 = k2_classification.drop(['classification', 'mission'], axis=1)  # Features
y_k2 = k2_classification['classification']  # Rótulos

from sklearn.model_selection import train_test_split

x_k2_treino, x_k2_teste, y_k2_treino, y_k2_teste = train_test_split(x_k2, y_k2, test_size=0.2, random_state=42)

In [2]:
import numpy as np

# Base treino não balanceada. Proporção
unique_values, counts = np.unique(y_k2_treino, return_counts=True)
prop = counts/y_k2_treino.size
print(unique_values,counts,prop)

['not planet' 'planet'] [ 245 1677] [0.12747138 0.87252862]


In [3]:
### montando base de treino balanceada

from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
x_k2_treino_bal, y_k2_treino_bal = smt.fit_resample(x_k2_treino, y_k2_treino)

ModuleNotFoundError: No module named 'imblearn'

In [ ]:
# base de treino balanceada. Proporção
unique_values, counts = np.unique(y_k2_treino_bal, return_counts=True)
prop = counts/y_k2_treino_bal.size
print(unique_values,counts,prop)

In [ ]:
from sklearn.naive_bayes import GaussianNB
### treinando modelo com dados nao balanceados e gerando previsoes com o modelo nao balanceado
modeloNB_nao_balanceado = GaussianNB()
modeloNB_nao_balanceado.fit(x_k2_treino,y_k2_treino)
y_k2_previsto_nao_balanceado = modeloNB_nao_balanceado.predict(x_k2_teste)

### treinando modelo com dados balanceados e gerando previsões com modelo balanceado
modeloNB_balanceado = GaussianNB()
modeloNB_balanceado.fit(x_k2_treino_bal,y_k2_treino_bal)
y_k2_previsto_balanceado = modeloNB_balanceado.predict(x_k2_teste)

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

In [ ]:
acc_nao_balanceado = accuracy_score(y_k2_teste,y_k2_previsto_nao_balanceado)
pre_nao_balanceado = precision_score(y_k2_teste,y_k2_previsto_nao_balanceado,pos_label='planet') 
rec_nao_balanceado = recall_score(y_k2_teste,y_k2_previsto_nao_balanceado,pos_label='planet')
f1_nao_balanceado = f1_score(y_k2_teste,y_k2_previsto_nao_balanceado,pos_label='planet')

acc_balanceado = accuracy_score(y_k2_teste,y_k2_previsto_balanceado)
pre_balanceado = precision_score(y_k2_teste,y_k2_previsto_balanceado,pos_label='planet')
rec_balanceado = recall_score(y_k2_teste,y_k2_previsto_balanceado,pos_label='planet')
f1_balanceado = f1_score(y_k2_teste,y_k2_previsto_balanceado,pos_label='planet')

### obs.: como nosso target não é 1 e 0, temos que informar às métricas precision, recall e f1_score quem é o target positivo. Por isso o argumento pos_label = 'planet'

In [ ]:
acc_nao_balanceado, acc_balanceado

In [ ]:
pre_nao_balanceado,pre_balanceado

In [ ]:
rec_nao_balanceado,rec_balanceado

In [ ]:
f1_nao_balanceado,f1_balanceado

In [ ]:
### tratamento de outliers
### é necessário transformar o target em númeors, pois a tecnica de identificação de outliers IsolationForest só trabalha com números
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y_k2_treino) 
### juntar dados de treinos
dados_para_analisar_outliers = np.column_stack((x_k2_treino, y_encoded))  
### identificar os outliers (supondo 5% de contaminação na base (ajustável))
from sklearn.ensemble import IsolationForest
clf = IsolationForest(contamination=0.01)
labels = clf.fit_predict(dados_para_analisar_outliers)
### rearrumando a base removendo outliers
dados_filtrados = dados_para_analisar_outliers[labels == 1]
x_k2_treino_filtrado = dados_filtrados[:, :-1]
y_k2_treino_filtrado = dados_filtrados[:, 1]
### Utilizamos o mesmo objeto de encode para retransformar os target numéricos em categoricos
y_k2_treino_filtrado = encoder.inverse_transform(y_k2_treino_filtrado.astype(int))

In [ ]:
x_k2_treino_filtrado

In [ ]:
### treinando modelo com dados com outliers removidos
modeloNB_limpo = GaussianNB()
modeloNB_limpo.fit(x_k2_treino_filtrado, y_k2_treino_filtrado)
y_k2_previsto_limpo = modeloNB_limpo.predict(x_k2_teste.to_numpy())

In [ ]:
valores_unicos, contagens = np.unique(y_k2_treino_filtrado, return_counts=True)
valores_unicos, contagens

In [ ]:
acc_limpo = accuracy_score(y_k2_teste,y_k2_previsto_limpo)
pre_limpo = precision_score(y_k2_teste,y_k2_previsto_limpo,pos_label='planet')
rec_limpo = recall_score(y_k2_teste,y_k2_previsto_limpo,pos_label='planet')
f1_limpo = f1_score(y_k2_teste,y_k2_previsto_limpo,pos_label='planet')

In [ ]:
acc_nao_balanceado,acc_limpo

In [ ]:
pre_nao_balanceado,pre_limpo

In [ ]:
rec_nao_balanceado,rec_limpo

In [ ]:
f1_nao_balanceado,f1_limpo